In [28]:
import warnings
warnings.filterwarnings('ignore')
import glob
from shutil import copyfile
import random
import cv2
import dlib
import numpy as np
random.seed(123)
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

In [29]:
import math
def get_landmarks(image):
    data={}
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
        # calculate the center of gravity
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        # calculate the distance from center from both axis.
        # this information is used to get the angle relative to center point.
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            # append x and y values
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            # calculate the euclidiean distance from center
            dist = np.linalg.norm(coornp-meannp)
            # append the distance to the feature vector
            landmarks_vectorised.append(dist)
            # apend the angle relative to the center of gravity.
            landmarks_vectorised.append(math.degrees(math.atan2(y,x)))
        data['landmarks_vectorised'] = landmarks_vectorised
    if len(detections) < 1:
        data['landmarks_vestorised'] = "error"
    return data

In [30]:
def get_files(emotion):
    files = glob.glob("dataset/{0}/*".format(emotion))
    random.shuffle(files)
    train = files[:int(len(files)*0.8)] #get first 80% of file list
    test = files[-int(len(files)*0.2):] #get last 20% of file list
    return train, test

In [31]:
def make_sets():
    training_data = []
    training_labels = []
    test_data = []
    test_labels = []
    train_files_list = [] # its a list of dict of trainfiles
    test_files_list = [] 
    for emotion in emotions:
        #print(" working on %s" %emotion)
        train, test = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in train:
            d = dict()
            d["name"]= item
            d["emotion"] = emotions.index(emotion)
            train_files_list.append(d)
            image = cv2.imread(item) #open image
            data = get_landmarks(image)
            if data['landmarks_vectorised'] == "error":
                print("no face detected on this one")
            else:
                training_data.append(data['landmarks_vectorised']) #append image array to training data list
                training_labels.append(emotions.index(emotion))
        for item in test:
            d = dict()
            d["name"]= item
            d["emotion"] = emotions.index(emotion)
            test_files_list.append(d)
            image = cv2.imread(item)
            data = get_landmarks(image)
            if data['landmarks_vectorised'] == "error":
                print("no face detected on this one")
            else:
                test_data.append(data['landmarks_vectorised'])
                test_labels.append(emotions.index(emotion))
    return training_data, training_labels, test_data, test_labels, train_files_list, test_files_list

In [32]:
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Landmark identifier. Set the filename to whatever you named the downloaded file

In [33]:
from sklearn.svm import SVC
from sklearn.metrics import log_loss
clf_lin_svm = SVC(kernel='linear', random_state=7, probability=True, tol=1e-3)

train_data, train_labels, test_data, test_labels, train_files_list, test_files_list = make_sets()

X_train = np.array(train_data) #Turn the training set into a numpy array for the classifier    
y_train = np.array(train_labels)

clf_lin_svm.fit(X_train, y_train)

X_test = np.array(test_data)
y_test = np.array(test_labels)
train_accuracy = clf_lin_svm.score(X_train, y_train)        
test_accuracy = clf_lin_svm.score(X_test, y_test)

In [40]:
X_test[1].reshape(1, -1).shape

(1, 268)

In [35]:
print("Train Accuracy Score for trail = {0} ".format(train_accuracy))
print("Test Accuracy Score for trail = {0} ".format(test_accuracy))

train_pred_proba_lin = clf_lin_svm.predict_proba(X_train)
print("Train log loss for trail = {0}".format(log_loss(train_labels, train_pred_proba_lin)))

test_pred_proba_lin =clf_lin_svm.predict_proba(X_test)
print("Test log loss for trail = {0}".format(log_loss(test_labels, test_pred_proba_lin)))

Train Accuracy Score for trail = 1.0 
Test Accuracy Score for trail = 0.7764705882352941 
Train log loss for trail = 0.2780392533170983
Test log loss for trail = 0.6067184668743413


In [43]:
clf_lin_svm.predict(X_test[3].reshape(1, -1))

array([7])

In [44]:
import pickle

In [45]:
s = pickle.dumps(clf_lin_svm)

In [50]:
import joblib

In [51]:
joblib.dump(clf_lin_svm, 'SVM_model.pkl')

['SVM_model.pkl']